# Pydantic

Pydantic is a data validation libarary for python
- Works with python type annotations, but rather than static type checking they are actively used at runtime for data validationand coversion
- provides built-in methods to serialize/deserialize models to/from JSON, dictionaries, etc
- langchain leverages Pydantic to create JSON Shecma describing function


In [34]:
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [3]:
import os
import openai

In [4]:
from typing import List
from pydantic import BaseModel, Field

In [7]:
class User:
  def __init__(self, name: str, age: int, email: str):
    self.name = name
    self.age = age
    self.email = email

In [8]:
foo = User(name = 'Joe', age = "haha", email = 'd@d.com')

In [10]:
foo.age ## works even though age is set to int

'haha'

In [13]:
class pUser(BaseModel):
  name: str
  age: int
  email:str

In [14]:
foo_p = pUser(name = 'Joe', age = "haha", email = 'd@d.com')  #doesnot works

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='haha', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [15]:
foo_p = pUser(name = 'Joe', age = 2, email = 'd@d.com')
foo_p.age #works

2

In [26]:
class Classs(BaseModel):
  students: List[pUser]

In [28]:
obj = Classs(
    students = [pUser(name = 'Joe', age = 12, email = 'd@d.com')]
)

obj ##nested pydantrics

Classs(students=[pUser(name='Joe', age=12, email='d@d.com')])

# Pydantic to OpenAi function definition

In [41]:
class WeatherSearch(BaseModel):
  'Call this with an airport code to get the weather at that airport'
  airport_code: str = Field(description = 'airport code to get weather forcast')

In [42]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [43]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [44]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather forcast',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [49]:
from langchain.chat_models import ChatOpenAI
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')

In [50]:
model = ChatOpenAI(openai_api_key = key)

In [57]:
model.invoke('What is the weather in Nepal today?', functions = [weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"VNKT"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 71, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e3bf8248-37b6-4179-82f4-1fa30d20c4b4-0')

In [58]:
model_with_function = model.bind(functions = [weather_function])

In [61]:
model_with_function.invoke('what is the weather at Nepal?')

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"VNKT"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-a02fe5dd-fa7e-4f96-8e74-aacfa3fbc23c-0')

In [65]:
model_with_forced_function = model.bind(functions = [weather_function], function_call = {"name": "WeatherSearch"})

model_with_forced_function.invoke('what is the weather in NEP?')

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"NEP"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 81, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b8141420-3fb9-4a4e-b52f-bd0503794b2e-0')

# Using in a chain

In [66]:
from langchain.prompts import ChatPromptTemplate


In [67]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful female assistant"),
    ("user", "{input}")
])

In [68]:
chain = prompt | model_with_forced_function

In [71]:
chain.invoke({"input": "what is the weather in Nep"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"NEP"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 86, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fe7c7862-1e62-4957-aab4-b6e4de95889f-0')

# Using multiple functions


In [81]:
class ArtistSearch(BaseModel):
  'Cal this to get the names of songs by a particular artist'
  artist_name: str = Field(description = "name of artist to look up")
  n: int = Field ( description = 'number of results')

In [82]:
function = convert_pydantic_to_openai_function(ArtistSearch)
function

{'name': 'ArtistSearch',
 'description': 'Cal this to get the names of songs by a particular artist',
 'parameters': {'properties': {'artist_name': {'description': 'name of artist to look up',
    'type': 'string'},
   'n': {'description': 'number of results', 'type': 'integer'}},
  'required': ['artist_name', 'n'],
  'type': 'object'}}

In [89]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch)
]

functions

[{'name': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather forcast',
     'type': 'string'}},
   'required': ['airport_code'],
   'type': 'object'}},
 {'name': 'ArtistSearch',
  'description': 'Cal this to get the names of songs by a particular artist',
  'parameters': {'properties': {'artist_name': {'description': 'name of artist to look up',
     'type': 'string'},
    'n': {'description': 'number of results', 'type': 'integer'}},
   'required': ['artist_name', 'n'],
   'type': 'object'}}]

In [93]:
model_with_function = model.bind(functions=functions)

In [95]:
model_with_function.invoke(' names of two songs by a taylor swift')

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":2}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 119, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f1bb829f-71d7-45fa-a905-1e3e9f546691-0')